In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import datetime
from dateutil.relativedelta import relativedelta


In [2]:
os.chdir('d:/CMAP')
ds_precip = xr.open_dataset('precip.mon.mean.nc') # ds_precip

#### cut new clim

In [4]:
# new_clim_ds
new_clim = ds_precip.sel(time=slice('1991-01-01','2020-12-01'))
new_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
Data variables:
    precip   (time, lat, lon) float32 ...

In [5]:
# Unit : mm/day -> mm/month
for i in range(len(new_clim.precip)):
    p_time = str(new_clim.time[i].values)[:10] # '1979-03-01'
    b = datetime.datetime.strptime(p_time,'%Y-%m-%d')
    nums_of_day = ((b + relativedelta(months=i))-datetime.timedelta(days=1)).day
    new_clim.precip[i] *= nums_of_day
precip = new_clim.precip

In [6]:
lon = new_clim.lon
lat = new_clim.lat
time = new_clim.time


length_precip= len(new_clim.time) # 360
glb_mean_lst = [] #glb_mean
time_lst = []

In [7]:
### add clim variable / len  30
clim_sel = new_clim['precip'].groupby('time.month').mean('time')
new_clim['new_clim'] = clim_sel
new_clim

<xarray.Dataset>
Dimensions:   (time: 360, lat: 72, lon: 144, month: 12)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon       (lon) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat       (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * month     (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip    (time, lat, lon) float32 ...
    new_clim  (month, lat, lon) float32 7.182 8.205 8.07 ... 13.07 13.2 13.6

In [8]:
### add anom variable
anom_lst =[]
for i in range(1,13):
    mon_group = new_clim.sel(time=new_clim['time.month']==i) # 각 월 값
    anom_sel = mon_group.precip - clim_sel[i-1]
    ## 나는 clim_sel[1] 이 2월 인거 알지만, 셀이 규칙이 맞지 않았을때 코드상에서   mon group도 1월이고 clim_sel도 1월인 값들을 계산해라! 를 짜고 싶어 
    anom_lst.append(anom_sel)
    
anom_cb = xr.concat(anom_lst,dim='time')
new_clim['new_anom'] = anom_cb
new_clim

c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


<xarray.Dataset>
Dimensions:   (time: 360, lat: 72, lon: 144, month: 12)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon       (lon) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat       (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * month     (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip    (time, lat, lon) float32 ...
    new_clim  (month, lat, lon) float32 7.182 8.205 8.07 ... 13.07 13.2 13.6
    new_anom  (time, lat, lon) float32 8.318 7.295 7.43 ... -13.07 -13.2 -13.6

In [ ]:
new_clim.to_netcdf('CMAP_new_clim_anom.nc')